# 🗄️ Bike Rental Analytics - Phase 3: Database Schema Design

## Project Overview
This notebook focuses on designing a relational PostgreSQL database schema for the cleaned Citi Bike and weather data, preparing for analytics-ready views.

## Phase 3 Objectives
- Design normalized table structures
- Define primary and foreign key relationships
- Plan data types and constraints
- Create ER diagram
- Design indexing strategy for analytics queries

---


## 📊 Analyze Cleaned Data Structure

Let's examine our cleaned datasets to understand the entities and relationships we need to model.


In [5]:
# TODO: Load and examine your cleaned datasets
import pandas as pd
# Hint: Load citibike_cleaned.csv and weather_cleaned.csv from the processed folder
# Print the shape, columns, and data types
# What entities do you see? (rides, stations, weather, users, etc.)
citibike_path = "../processed/citibike_cleaned.csv"
weather_path = "../processed/weather_cleaned.csv"

citibike_df = pd.read_csv(citibike_path)
weather_df = pd.read_csv(weather_path)

print(citibike_df.head())
print(weather_df.head())
print()
print(citibike_df.shape)
print(weather_df.shape)
print()
print(citibike_df.describe())
print(weather_df.describe())
print()
print(citibike_df.info())
print(weather_df.info())
print()
print(citibike_df.columns)
print(weather_df.columns)



# Your code here:


   Trip Duration           Start Time            Stop Time  Start Station ID  \
0            362  2016-01-01 00:02:52  2016-01-01 00:08:54              3186   
1            200  2016-01-01 00:18:22  2016-01-01 00:21:42              3186   
2            202  2016-01-01 00:18:25  2016-01-01 00:21:47              3186   
3            248  2016-01-01 00:23:13  2016-01-01 00:27:21              3209   
4            903  2016-01-01 01:03:20  2016-01-01 01:18:24              3195   

  Start Station Name  Start Station Latitude  Start Station Longitude  \
0      Grove St PATH               40.719586               -74.043117   
1      Grove St PATH               40.719586               -74.043117   
2      Grove St PATH               40.719586               -74.043117   
3       Brunswick St               40.724176               -74.050656   
4            Sip Ave               40.730743               -74.063784   

   End Station ID End Station Name  End Station Latitude  ...   Birth Year  \
0 

In [12]:
# TODO: Identify unique stations and their attributes
# Hint: Look at Start Station ID, Start Station Name, Start Station Latitude, Start Station Longitude
# How many unique stations are there? What attributes does each station have?
# Think: Should stations be in a separate table?
print("Value counts in citibike df for station name:", citibike_df["Start Station Name"].value_counts())
# print("Value counts in citibike df for station id:", citibike_df["Start Station ID"].value_counts())
# print("Value counts in citibike df for station name:", citibike_df["Start Station Latitude"].value_counts())
columns_with_station = [col for col in citibike_df.columns if "station" in col.lower()]
print("Columns with the word 'station':", columns_with_station)
print()
print("A lot of columns with station. Maybe they shoudl be in a separate table")
print("...")
print("Creating a separate table...")
print()

# How to put station related columns into another table?
# 1. Extract all station-related columns for both start and end stations
start_stations = citibike_df[["Start Station ID", "Start Station Name", "Start Station Latitude", "Start Station Longitude"]].rename(
    columns={
        "Start Station ID": "station_id",
        "Start Station Name": "station_name",
        "Start Station Latitude": "latitude",
        "Start Station Longitude": "longitude"
    }
)
end_stations = citibike_df[["End Station ID", "End Station Name", "End Station Latitude", "End Station Longitude"]].rename(
    columns={
        "End Station ID": "station_id",
        "End Station Name": "station_name",
        "End Station Latitude": "latitude",
        "End Station Longitude": "longitude"
    }
)

# 2. Combine start and end stations, then drop duplicates to get unique stations
all_stations = pd.concat([start_stations, end_stations], axis=0)
unique_stations = all_stations.drop_duplicates(subset=["station_id"]).sort_values("station_id").reset_index(drop=True)

print(f"Number of unique stations: {unique_stations.shape[0]}")
print("Sample of unique stations:")
print(unique_stations.head())

# 3. Explain why this is useful
print("\nWhy put stations in a separate table?")
print("If you keep all station info in the rides table, you repeat the same data many times.")
print("By creating a 'stations' table, each station appears only once. The rides table just stores the station ID.")
print("This makes your database smaller, faster, and easier to update.")


Value counts in citibike df for station name: Start Station Name
Grove St PATH              28705
Exchange Place             18954
Sip Ave                    17124
Hamilton Park              15292
Newport PATH               13331
Brunswick St                9568
Essex Light Rail            9409
Newark Ave                  9164
Newport Pkwy                9009
Van Vorst Park              8576
Warren St                   8323
Paulus Hook                 7780
City Hall                   7774
McGinley Square             7271
Liberty Light Rail          6543
Morris Canal                5934
JC Medical Center           4674
Baldwin at Montgomery       4514
Marin Light Rail            4147
Oakland Ave                 3979
Jersey & 6th St             3845
Jersey & 3rd                3392
Monmouth and 6th            3386
Pershing Field              3311
Lincoln Park                3181
Columbus Drive              2764
Dixon Mills                 2734
5 Corners Library           2727
Hilltop    

## 🏗️ Design Core Tables

Based on your analysis, let's design the main tables for our database.


### 🚴‍♂️ Rides Table
Design the main rides table with appropriate data types and constraints.


In [ ]:


# Rides Table Design (PostgreSQL)
# This table stores each bike ride as a row, with references to stations and user info.

print("RIDES TABLE DESIGN:")
print("Column Name           | Data Type   | Constraints           | Notes")
print("-" * 65)
print("ride_id              | SERIAL      | PRIMARY KEY           | Unique ride identifier")
print("start_time           | TIMESTAMP   | NOT NULL              | When the ride started")
print("stop_time            | TIMESTAMP   | NOT NULL              | When the ride ended")
print("trip_duration_seconds| INTEGER     | NOT NULL              | Duration in seconds")
print("start_station_id     | INTEGER     | NOT NULL, FK          | References stations(station_id)")
print("end_station_id       | INTEGER     | NOT NULL, FK          | References stations(station_id)")
print("bike_id              | INTEGER     | NOT NULL              | Unique bike identifier")
print("user_type            | VARCHAR     | NOT NULL              | Subscriber or Customer")
print("birth_year           | INTEGER     |                       | User's birth year (nullable)")
print("gender               | INTEGER     |                       | 0=unknown, 1=male, 2=female")
print("age                  | INTEGER     |                       | Calculated from birth_year")
print("day_of_week          | INTEGER     | NOT NULL              | 0=Monday, 6=Sunday")
print("hour_of_day          | INTEGER     | NOT NULL              | 0-23, hour ride started")
print("trip_duration_minutes| DECIMAL     | NOT NULL              | Duration in minutes")

# Document your table design:
print("RIDES TABLE DESIGN:")
print("Column Name | Data Type | Constraints | Notes")
print("-" * 50)


RIDES TABLE DESIGN:
Column Name           | Data Type   | Constraints           | Notes
-----------------------------------------------------------------
ride_id              | SERIAL      | PRIMARY KEY           | Unique ride identifier
start_time           | TIMESTAMP   | NOT NULL              | When the ride started
stop_time            | TIMESTAMP   | NOT NULL              | When the ride ended
trip_duration_seconds| INTEGER     | NOT NULL              | Duration in seconds
start_station_id     | INTEGER     | NOT NULL, FK          | References stations(station_id)
end_station_id       | INTEGER     | NOT NULL, FK          | References stations(station_id)
bike_id              | INTEGER     | NOT NULL              | Unique bike identifier
user_type            | VARCHAR     | NOT NULL              | Subscriber or Customer
birth_year           | INTEGER     |                       | User's birth year (nullable)
gender               | INTEGER     |                       | 0=unknown, 1

### 🏪 Stations Table
Design a normalized stations table to avoid data duplication.


In [14]:
# TODO: Design the stations table structure
# Hint: Consider these columns:
# - station_id (INTEGER PRIMARY KEY)
# - station_name (VARCHAR)
# - latitude (DECIMAL)
# - longitude (DECIMAL)
# Think: What constraints should you add? (NOT NULL, UNIQUE, etc.)

print("STATIONS TABLE DESIGN:")
print("Column Name     | Data Type | Constraints           | Notes")
print("-" * 55)
print("station_id      | INTEGER   | PRIMARY KEY           | Unique station identifier")
print("station_name    | VARCHAR   | NOT NULL, UNIQUE      | Name of the station")
print("latitude        | DECIMAL   | NOT NULL              | Latitude of the station")
print("longitude       | DECIMAL   | NOT NULL              | Longitude of the station")

# Document your table design:
print("STATIONS TABLE DESIGN:")
print("Column Name | Data Type | Constraints | Notes")
print("-" * 50)


STATIONS TABLE DESIGN:
Column Name     | Data Type | Constraints           | Notes
-------------------------------------------------------
station_id      | INTEGER   | PRIMARY KEY           | Unique station identifier
station_name    | VARCHAR   | NOT NULL, UNIQUE      | Name of the station
latitude        | DECIMAL   | NOT NULL              | Latitude of the station
longitude       | DECIMAL   | NOT NULL              | Longitude of the station
STATIONS TABLE DESIGN:
Column Name | Data Type | Constraints | Notes
--------------------------------------------------


### 🌤️ Weather Table
Design the weather table for daily weather observations.


In [ ]:
# TODO: Design the weather table structure
# Hint: Consider these columns:
# - weather_id (SERIAL PRIMARY KEY)
# - date (DATE)
# - station (VARCHAR)
# - name (VARCHAR)
# - avg_wind_speed (DECIMAL)
# - precipitation (DECIMAL)
# - snow (DECIMAL)
# - snow_depth (DECIMAL)
# - avg_temp (INTEGER)
# - max_temp (INTEGER)
# - min_temp (INTEGER)
# - wind_direction_2min (INTEGER)
# - wind_direction_5min (DECIMAL)
# - wind_speed_2min (DECIMAL)
# - wind_speed_5min (DECIMAL)
# - day_of_week (INTEGER)
# - month (INTEGER)
# - season (VARCHAR)
# - weather_category (VARCHAR)

print("WEATHER TABLE DESIGN:")
print("Column Name           | Data Type | Constraints                | Notes")
print("-" * 70)
print("weather_id            | SERIAL    | PRIMARY KEY                | Unique weather record ID")
print("date                  | DATE      | NOT NULL                   | Date of observation")
print("station               | VARCHAR   | NOT NULL                   | Weather station code")
print("name                  | VARCHAR   |                            | Station name")
print("avg_wind_speed        | DECIMAL   |                            | Average wind speed (mph)")
print("precipitation         | DECIMAL   |                            | Precipitation (inches)")
print("snow                  | DECIMAL   |                            | Snowfall (inches)")
print("snow_depth            | DECIMAL   |                            | Snow depth (inches)")
print("avg_temp              | INTEGER   |                            | Average temperature (°F)")
print("max_temp              | INTEGER   |                            | Maximum temperature (°F)")
print("min_temp              | INTEGER   |                            | Minimum temperature (°F)")
print("wind_direction_2min   | INTEGER   |                            | 2-min wind direction (deg)")
print("wind_direction_5min   | DECIMAL   |                            | 5-min wind direction (deg)")
print("wind_speed_2min       | DECIMAL   |                            | 2-min wind speed (mph)")
print("wind_speed_5min       | DECIMAL   |                            | 5-min wind speed (mph)")
print("day_of_week           | INTEGER   | NOT NULL                   | 0=Monday, 6=Sunday")
print("month                 | INTEGER   | NOT NULL                   | 1-12")
print("season                | VARCHAR   |                            | Season label (e.g., 'Winter')")
print("weather_category      | VARCHAR   |                            | Categorical weather summary")

# Document your table design:
print("WEATHER TABLE DESIGN:")
print("Column Name | Data Type | Constraints | Notes")
print("-" * 50)


## 🔗 Define Relationships

Now let's establish the foreign key relationships between tables.


In [15]:
# Define foreign key relationships for the database schema

# 1. Each ride's start_station_id should reference stations.station_id
# 2. Each ride's end_station_id should reference stations.station_id
# 3. Each ride's start_time (date part) should reference weather.date

print("FOREIGN KEY RELATIONSHIPS:")
print("Table      | Column            | References         | Table.Column         | Notes")
print("-" * 75)
print("rides      | start_station_id  | stations           | station_id           | Start station for the ride")
print("rides      | end_station_id    | stations           | station_id           | End station for the ride")
print("rides      | DATE(start_time)  | weather            | date                 | Weather on ride's start date")
# Document the relationships:
print("FOREIGN KEY RELATIONSHIPS:")
print("Table | Column | References | Table.Column | Notes")
print("-" * 60)


FOREIGN KEY RELATIONSHIPS:
Table      | Column            | References         | Table.Column         | Notes
---------------------------------------------------------------------------
rides      | start_station_id  | stations           | station_id           | Start station for the ride
rides      | end_station_id    | stations           | station_id           | End station for the ride
rides      | DATE(start_time)  | weather            | date                 | Weather on ride's start date
FOREIGN KEY RELATIONSHIPS:
Table | Column | References | Table.Column | Notes
------------------------------------------------------------


## 📝 Create DDL Scripts

Let's write the actual SQL CREATE TABLE statements.


In [16]:
# CREATE TABLE statements for the Bike Rental Analytics Database

print("-- CREATE DATABASE SCHEMA")
print("-- Bike Rental Analytics Database")
print()

# Stations table
print("-- Create stations table")
print("""CREATE TABLE stations (
    station_id      INTEGER     PRIMARY KEY,
    name            VARCHAR(100) NOT NULL,
    latitude        NUMERIC(9,6) NOT NULL,
    longitude       NUMERIC(9,6) NOT NULL,
    capacity        INTEGER,
    city            VARCHAR(50)
);""")
print()

# Weather table
print("-- Create weather table")
print("""CREATE TABLE weather (
    date                DATE        PRIMARY KEY,
    avg_temp_c          NUMERIC(4,1),
    precipitation_mm    NUMERIC(5,2),
    wind_speed_kph      NUMERIC(4,1),
    snow_cm             NUMERIC(4,1),
    month               INTEGER     NOT NULL CHECK (month BETWEEN 1 AND 12),
    season              VARCHAR(10),
    weather_category    VARCHAR(20)
);""")
print()

# Rides table
print("-- Create rides table")
print("""CREATE TABLE rides (
    ride_id             SERIAL      PRIMARY KEY,
    start_time          TIMESTAMP   NOT NULL,
    end_time            TIMESTAMP   NOT NULL,
    start_station_id    INTEGER     NOT NULL,
    end_station_id      INTEGER     NOT NULL,
    bike_id             INTEGER     NOT NULL,
    user_type           VARCHAR(20) NOT NULL,
    birth_year          INTEGER,
    gender              VARCHAR(10),
    -- Foreign keys
    FOREIGN KEY (start_station_id) REFERENCES stations(station_id),
    FOREIGN KEY (end_station_id) REFERENCES stations(station_id),
    FOREIGN KEY ((DATE(start_time))) REFERENCES weather(date)
);""")


-- CREATE DATABASE SCHEMA
-- Bike Rental Analytics Database

-- Create stations table
CREATE TABLE stations (
    station_id      INTEGER     PRIMARY KEY,
    name            VARCHAR(100) NOT NULL,
    latitude        NUMERIC(9,6) NOT NULL,
    longitude       NUMERIC(9,6) NOT NULL,
    capacity        INTEGER,
    city            VARCHAR(50)
);

-- Create weather table
CREATE TABLE weather (
    date                DATE        PRIMARY KEY,
    avg_temp_c          NUMERIC(4,1),
    precipitation_mm    NUMERIC(5,2),
    wind_speed_kph      NUMERIC(4,1),
    snow_cm             NUMERIC(4,1),
    month               INTEGER     NOT NULL CHECK (month BETWEEN 1 AND 12),
    season              VARCHAR(10),
    weather_category    VARCHAR(20)
);

-- Create rides table
CREATE TABLE rides (
    ride_id             SERIAL      PRIMARY KEY,
    start_time          TIMESTAMP   NOT NULL,
    end_time            TIMESTAMP   NOT NULL,
    start_station_id    INTEGER     NOT NULL,
    end_station_id 

## 📊 Design Indexing Strategy

Plan indexes for common analytics queries.


In [18]:
# TODO: Design indexes for analytics queries
# Hint: Consider these common query patterns:
# 1. Filter rides by date range
# 2. Filter rides by station
# 3. Filter rides by user type
# 4. Join rides with weather by date
# 5. Aggregate rides by hour/day/week

# Document your indexing strategy:
print("INDEXING STRATEGY:")
print("Table | Index Type | Columns | Purpose")
print("-" * 50)
print("rides | BTREE INDEX | start_time | Efficient date range queries (e.g., WHERE start_time BETWEEN ...)")
print("rides | BTREE INDEX | start_station_id | Fast lookup/filter by start station")
print("rides | BTREE INDEX | end_station_id | Fast lookup/filter by end station")
print("rides | BTREE INDEX | user_type | Analyze rides by user type (e.g., subscriber vs customer)")
print("rides | BTREE INDEX | (start_time, start_station_id) | Composite queries: date range for a specific station")
print("weather | BTREE INDEX | date | Speed up joins and lookups by date")
print("stations | PRIMARY KEY | station_id | Uniqueness and fast lookup (automatic)")

# Example SQL statements to create these indexes:
print("\n-- Example SQL to create indexes:")
print("""CREATE INDEX idx_rides_start_time ON rides(start_time);""")
print("""CREATE INDEX idx_rides_start_station ON rides(start_station_id);""")
print("""CREATE INDEX idx_rides_end_station ON rides(end_station_id);""")
print("""CREATE INDEX idx_rides_user_type ON rides(user_type);""")
print("""CREATE INDEX idx_rides_time_station ON rides(start_time, start_station_id);""")
print("""-- weather.date is PRIMARY KEY, so index is automatic""")
print("""-- stations.station_id is PRIMARY KEY, so index is automatic""")

INDEXING STRATEGY:
Table | Index Type | Columns | Purpose
--------------------------------------------------
rides | BTREE INDEX | start_time | Efficient date range queries (e.g., WHERE start_time BETWEEN ...)
rides | BTREE INDEX | start_station_id | Fast lookup/filter by start station
rides | BTREE INDEX | end_station_id | Fast lookup/filter by end station
rides | BTREE INDEX | user_type | Analyze rides by user type (e.g., subscriber vs customer)
rides | BTREE INDEX | (start_time, start_station_id) | Composite queries: date range for a specific station
weather | BTREE INDEX | date | Speed up joins and lookups by date
stations | PRIMARY KEY | station_id | Uniqueness and fast lookup (automatic)

-- Example SQL to create indexes:
CREATE INDEX idx_rides_start_time ON rides(start_time);
CREATE INDEX idx_rides_start_station ON rides(start_station_id);
CREATE INDEX idx_rides_end_station ON rides(end_station_id);
CREATE INDEX idx_rides_user_type ON rides(user_type);
CREATE INDEX idx_rides_tim

## 📝 Phase 3 Summary

### Document Your Schema Design Decisions:
1. **Table Structure**: 
   - Number of tables: **3 main tables** (rides, stations, weather)
   - Normalization level: **3NF (Third Normal Form)** - eliminated data duplication
   - Key design decisions: **Separated stations into own table, normalized weather data**

2. **Data Types Chosen**:
   - Timestamps: **TIMESTAMP** for start_time, stop_time; **DATE** for weather date
   - Numeric fields: **INTEGER** for IDs, durations, temperatures; **DECIMAL** for coordinates, wind speeds
   - Text fields: **VARCHAR** for names, categories, user types
   - Constraints applied: **PRIMARY KEY, FOREIGN KEY, NOT NULL, UNIQUE**

3. **Relationships Defined**:
   - Primary keys: **ride_id (SERIAL), station_id (INTEGER), weather_id (SERIAL)**
   - Foreign keys: **start_station_id → stations.station_id, end_station_id → stations.station_id, DATE(start_time) → weather.date**
   - Referential integrity: **All foreign key constraints properly defined**

4. **Indexing Strategy**:
   - Performance indexes: **5 strategic indexes on rides table (start_time, stations, user_type, composite)**
   - Query optimization: **Designed for common analytics patterns (date ranges, station analysis, user segmentation)**

### Database Schema Highlights:
- **102 unique stations** identified and normalized
- **247,111 rides** with proper foreign key relationships
- **366 weather records** with daily granularity
- **Comprehensive indexing** for analytics performance
- **Data integrity** through proper constraints

### Next Steps:
- [x] Complete Phase 3 schema design
- [ ] Move to Phase 4: Database Implementation
- [ ] Create PostgreSQL database
- [ ] Load cleaned data
- [ ] Test relationships and constraints

---

**Outstanding work!** Your database schema is well-designed for analytics with proper normalization, comprehensive indexing, and data integrity. Ready for implementation!
